In [18]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Database configuration from environment variables
db_config = {
    "host": os.getenv("DB_HOST"),
    "port": os.getenv("DB_PORT"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
    "dbname": os.getenv("DB_NAME")
}

def read_data(table_name):
    """
    Read data from the specified PostgreSQL table and return it as a DataFrame.
    """
    try:
        # Connect to the PostgreSQL database
        conn = psycopg2.connect(**db_config)
        
        # Create a cursor object
        cursor = conn.cursor()
        
        # Execute a query to fetch data
        query = f"SELECT * FROM {table_name};"
        cursor.execute(query)
        
        # Fetch all rows from the executed query
        rows = cursor.fetchall()
        
        # Convert the data to a Pandas DataFrame
        columns = [desc[0] for desc in cursor.description]  # Get column names
        df = pd.DataFrame(rows, columns=columns)
        
        # Convert the 'date' column to datetime
        df['date'] = pd.to_datetime(df['date'], errors='coerce')  # Convert to datetime
        
        return df  # Return the DataFrame
        
    except Exception as e:
        print(f"Error reading data from PostgreSQL: {e}")
        return None  # Return None in case of an error
    finally:
        if conn:
            cursor.close()
            conn.close()

# Call the function to read data from the specified table and assign it to df
df = read_data("historical_stock_data")

# Now df is defined and can be used outside the function
if df is None:
    print("No data was retrieved.")

In [19]:
df.dtypes

date          datetime64[ns]
open                 float64
high                 float64
low                  float64
close                float64
adj_close            float64
volume                 int64
stock_name            object
dtype: object

In [20]:
df.head()

,date,open,high,low,close,adj_close,volume,stock_name
0,2020-01-02,72.796013,75.087502,75.150002,73.797501,74.059998,135480400,AAPL
1,2020-01-03,72.088303,74.357498,75.144997,74.125000,74.287498,146322800,AAPL
2,2020-01-06,72.662720,74.949997,74.989998,73.187500,73.447502,118387200,AAPL
3,2020-01-07,72.320992,74.597504,75.224998,74.370003,74.959999,108872000,AAPL
4,2020-01-08,73.484337,75.797501,76.110001,74.290001,74.290001,132079200,AAPL


In [21]:
df = df.sort_values(by='date').reset_index(drop=True)
df.head()

,date,open,high,low,close,adj_close,volume,stock_name
0,2020-01-02,72.796013,75.087502,75.150002,73.797501,74.059998,135480400,AAPL
1,2020-01-02,153.938171,160.619995,160.729996,158.330002,158.779999,22622100,MSFT
2,2020-01-02,3257.850098,3257.850098,3258.139893,3235.530029,3244.669922,3459930000,^GSPC
3,2020-01-02,8891.750000,8891.750000,8901.500000,8769.500000,8776.750000,416637,NQ=F
4,2020-01-02,5.972712,5.997750,5.997750,5.918000,5.968750,237536000,NVDA
